# Baseline Injury Risk Prediction Model (30-Day Horizon)

### A Reproducible End-to-End Pipeline from Raw Injury Logs to Probabilistic Risk Scores


In [9]:
import os
import pandas as pd
from supabase import create_client
from dotenv import load_dotenv

load_dotenv()
supabase = create_client(os.getenv("SUPABASE_URL"), os.getenv("SUPABASE_KEY"))

# Pull only what we need for baseline
COLS = "injury_id,player_id,team_id,injury_date,acquired,relinquished,body_region,diagnosis,return_status"
CHUNK = 1000

def fetch_injuries():
    rows = []
    start = 0

    while True:
        resp = (
            supabase.table("injuries")
            .select(COLS)
            .order("injury_date", desc=False)   # stable paging
            .range(start, start + CHUNK - 1)
            .execute()
        )

        batch = resp.data or []
        if not batch:
            break

        rows.extend(batch)
        print(f"Fetched {len(rows)} rows so far...")

        if len(batch) < CHUNK:
            break

        start += CHUNK

    df = pd.DataFrame(rows)
    df["injury_date"] = pd.to_datetime(df["injury_date"], errors="coerce")

    df.to_csv("injuries_raw.csv", index=False)
    print("Saved -> injuries_raw.csv")
    print(df.head())
    print("Rows:", len(df), "Players:", df["player_id"].nunique())



## Data Cleaning and Injury-Start Event Selection

In this step, I load the raw extract (`injuries_raw.csv`) and prepare a clean set of **injury start events** for modeling.

### Cleaning
- Remove rows missing `player_id` or `injury_date`
- Convert `injury_date` to a standard datetime format

### Injury Start Definition
For consistency, I only keep rows representing the **start of an injury**.
Based on the dataset semantics, an injury start is defined as:
- `acquired = True` (player is placed on the injury list)

The resulting dataset (`inj_start`) is the base table used for labeling and feature engineering.


In [19]:
import pandas as pd

df = pd.read_csv("injuries_raw.csv")
df["injury_date"] = pd.to_datetime(df["injury_date"], errors="coerce")

# 1) drop unusable rows
df = df[df["player_id"].notna()]
df = df[df["injury_date"].notna()]

# 2) keep ONLY injury-start rows (placed on injury list)
inj_start = df[df["acquired"] == True].copy()

print("inj_start rows:", len(inj_start))
print("inj_start players:", inj_start["player_id"].nunique())
print(inj_start[["player_id","injury_date","body_region","diagnosis","acquired","relinquished"]].head(10))


inj_start rows: 17309
inj_start players: 2445
    player_id injury_date body_region diagnosis  acquired  relinquished
5     76127.0  1962-03-24         NaN       NaN      True         False
6     76127.0  1962-03-31         NaN       NaN      True         False
8     76708.0  1962-11-06         NaN       NaN      True         False
12    76865.0  1969-10-28         NaN       NaN      True         False
17    77243.0  1971-11-08         NaN       NaN      True         False
23    76558.0  1972-12-16         NaN       NaN      True         False
25    78069.0  1973-02-24         NaN       NaN      True         False
26    76192.0  1973-03-19         NaN       NaN      True         False
27    77147.0  1973-11-07         NaN       NaN      True         False
29    76505.0  1973-12-20         NaN       NaN      True         False


### Handling Missing Categorical Data

`body_region` and `diagnosis` contain a substantial number of missing values.  
Rather than dropping these rows or imputing potentially misleading values, missing entries are treated as their own category (`"unknown"`).

This approach preserves data volume and allows the model to learn whether **missingness itself carries predictive signal**.


In [23]:
# Treat missing text as its own category (on injury-start events only)
inj_start["body_region"] = inj_start["body_region"].fillna("unknown")
inj_start["diagnosis"] = inj_start["diagnosis"].fillna("unknown")

# Quick sanity check
print(inj_start["body_region"].value_counts().head())
print(inj_start["diagnosis"].value_counts().head())


body_region
unknown    17307
knee           1
neck           1
Name: count, dtype: int64
diagnosis
unknown    17309
Name: count, dtype: int64


In [12]:
print(df.columns.tolist())



['injury_id', 'player_id', 'team_id', 'injury_date', 'acquired', 'relinquished', 'body_region', 'diagnosis', 'return_status']


## Label Construction and Feature Engineering

This step transforms injury start events into a modeling-ready dataset.

### Injury Start Alignment
- Only injury **start** events are retained (`acquired = True`)
- Events are sorted chronologically per player to preserve temporal order

### Label Definition (30-Day Horizon)
For each injury event, a binary label is created indicating whether the same player
sustains another injury within the next **30 days**:
- `1` = reinjury occurs within 30 days
- `0` = no reinjury within that window

### Feature Engineering
All features are computed using information available **at the time of injury**:
- `prior_injuries_count`: number of previous injury starts for the player
- `days_since_last_injury`: time since the previous injury (−1 if none)

### Final Dataset
The resulting dataset includes injury context, engineered features, and the target label.
Missing categorical values (`body_region`, `diagnosis`) are encoded as `"unknown"`.


In [24]:
import pandas as pd

# 1) parse dates (safe to keep even if already parsed)
df["injury_date"] = pd.to_datetime(df["injury_date"], errors="coerce")

# 2) keep "injury start" events only (acquired=True) + valid ids/dates
inj_start = df[
    df["player_id"].notna()
    & df["injury_date"].notna()
    & (df["acquired"] == True)
].copy()

inj_start = inj_start.sort_values(["player_id", "injury_date"])

# 3) define prediction horizon
Y = 30  # days

# 4) label: does another injury happen within Y days?
inj_start["next_injury_date"] = inj_start.groupby("player_id")["injury_date"].shift(-1)

inj_start["label_next_injury_within_Y"] = (
    (inj_start["next_injury_date"] - inj_start["injury_date"]).dt.days.between(1, Y)
).astype(int)

# 5) features: injury history known at this injury date
inj_start["prior_injuries_count"] = inj_start.groupby("player_id").cumcount()

inj_start["days_since_last_injury"] = (
    inj_start["injury_date"] - inj_start.groupby("player_id")["injury_date"].shift(1)
).dt.days.fillna(-1)

# 6) modeling dataset
dataset = inj_start[[
    "player_id",
    "injury_date",
    "body_region",
    "diagnosis",
    "prior_injuries_count",
    "days_since_last_injury",
    "label_next_injury_within_Y"
]].copy()

dataset["body_region"] = dataset["body_region"].fillna("unknown")
dataset["diagnosis"] = dataset["diagnosis"].fillna("unknown")

print("dataset columns:", dataset.columns.tolist())
print("label rate:", dataset["label_next_injury_within_Y"].mean())
dataset.head()


dataset columns: ['player_id', 'injury_date', 'body_region', 'diagnosis', 'prior_injuries_count', 'days_since_last_injury', 'label_next_injury_within_Y']
label rate: 0.30920330463920503


,player_id,injury_date,body_region,diagnosis,prior_injuries_count,days_since_last_injury,label_next_injury_within_Y
446,2.0,1985-11-22,unknown,unknown,0,-1.0,0
2423,3.0,1997-02-19,unknown,unknown,0,-1.0,0
3163,3.0,1998-04-20,unknown,unknown,1,425.0,0
3888,3.0,1999-12-27,unknown,unknown,2,616.0,0
4542,3.0,2001-01-19,unknown,unknown,3,389.0,0


> **Note:** This performance reflects a simple, interpretable baseline and is expected given limited feature availability.


## Baseline Model Training and Evaluation

This step trains an interpretable baseline classifier to estimate the probability of a reinjury within the **30-day horizon**.

### Time-Based Train/Test Split
- The dataset is sorted by `injury_date`
- The first 80% of events are used for training
- The most recent 20% are held out for testing

### Preprocessing
A preprocessing pipeline is applied:
- Numeric features (`prior_injuries_count`, `days_since_last_injury`) are passed through unchanged
- Categorical features (`body_region`, `diagnosis`) are one-hot encoded  
  (`handle_unknown="ignore"` ensures unseen categories in test data do not cause errors)

### Model Choice
A **Logistic Regression** model is used as a baseline because it is:
- Fast and simple
- Interpretable
- Produces probabilistic outputs  
Class imbalance is handled using `class_weight="balanced"`.

### Evaluation
Performance is measured with **ROC AUC**, which is:
- Threshold-independent (does not depend on choosing a probability cutoff)
- Appropriate for imbalanced binary classification


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# 1) Sort by time FIRST (so split is truly time-based)
dataset = dataset.sort_values("injury_date").reset_index(drop=True)

# 2) Features / label
X = dataset[["prior_injuries_count", "days_since_last_injury", "body_region", "diagnosis"]]
y = dataset["label_next_injury_within_Y"]

# 3) Time-based split
split = int(0.8 * len(dataset))
X_train, X_test = X.iloc[:split], X.iloc[split:]
y_train, y_test = y.iloc[:split], y.iloc[split:]

num_cols = ["prior_injuries_count", "days_since_last_injury"]
cat_cols = ["body_region", "diagnosis"]

preprocess = ColumnTransformer(
    [
        ("num", "passthrough", num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ]
)

model = LogisticRegression(max_iter=1000, class_weight="balanced")

pipe = Pipeline([
    ("prep", preprocess),
    ("clf", model)
])

pipe.fit(X_train, y_train)

proba_test = pipe.predict_proba(X_test)[:, 1]
print("ROC AUC:", roc_auc_score(y_test, proba_test))


ROC AUC: 0.6253417137663623


## Test-Set Predictions (Feature View)

This step builds a compact results table for the **test set** using the feature matrix (`X_test`) and then adds:

- `injury_date` for time context (pulled from the corresponding test rows in `dataset`)
- `actual` as the ground-truth label (`y_test`)
- `predicted_risk` as the model’s estimated probability of a reinjury within 30 days

The table is then sorted by `predicted_risk` to display the **top 10 highest-risk predictions**, which supports qualitative inspection of model ranking behavior.


In [27]:
# Add predictions to test set
showcase = X_test.copy()
showcase["injury_date"] = dataset.iloc[split:]["injury_date"].values
showcase["actual"] = y_test.values
showcase["predicted_risk"] = pipe.predict_proba(X_test)[:, 1]

# Sort by highest predicted risk
showcase = showcase.sort_values("predicted_risk", ascending=False)

showcase.head(10)


,prior_injuries_count,days_since_last_injury,body_region,diagnosis,injury_date,actual,predicted_risk
16936,48,24.0,unknown,unknown,2023-01-20,0,0.881054
17085,48,57.0,unknown,unknown,2023-03-05,0,0.878581
16835,47,10.0,unknown,unknown,2022-12-27,1,0.877349
16528,46,6.0,unknown,unknown,2022-06-08,0,0.872760
16788,46,7.0,unknown,unknown,2022-12-17,1,0.872681
16757,45,5.0,unknown,unknown,2022-12-10,1,0.867781
16527,45,48.0,unknown,unknown,2022-06-02,1,0.864245
17187,44,7.0,unknown,unknown,2023-03-24,0,0.862386
16483,44,9.0,unknown,unknown,2022-04-15,0,0.862218
16718,44,18.0,unknown,unknown,2022-12-05,1,0.861458


## Inspecting the Highest-Risk Prediction

This step extracts the **single highest-risk case** from the test set after sorting by predicted risk.

By inspecting this row in detail, we can:
- Understand which feature values are associated with the highest predicted risk
- Compare the model’s prediction (`predicted_risk`, `predicted_label`) to the true outcome
- Perform a qualitative sanity check on model behavior

This does **not** prove correctness, but helps interpret and validate the model’s risk ranking.


In [28]:
row = showcase.iloc[0]
row


prior_injuries_count                       48
days_since_last_injury                   24.0
body_region                           unknown
diagnosis                             unknown
injury_date               2023-01-20 00:00:00
actual                                      0
predicted_risk                       0.881054
Name: 16936, dtype: object

## Test-Set Risk Showcase (Top Predicted Cases)

This step creates a presentation table for the **test set** by combining:
- the original test-set rows (including the true label), and
- the model’s predicted probabilities.

### Predicted Risk
`predicted_risk` is the model’s estimated probability of **another injury within 30 days** (i.e., `P(label = 1)`).

### Optional Binary Prediction
A simple cutoff is applied for interpretation:
- `predicted_label = 1` if `predicted_risk ≥ 0.5`
- `predicted_label = 0` otherwise  

> Note: the 0.5 threshold is a default; evaluation is primarily based on ROC AUC (threshold-independent).

### Output
The table is sorted by `predicted_risk` to display the **top 10 highest-risk cases**, allowing qualitative inspection of model behavior.


In [30]:
showcase = dataset.iloc[split:].copy()

showcase["predicted_risk"] = pipe.predict_proba(X_test)[:, 1]

# Optional: binary prediction
showcase["predicted_label"] = (showcase["predicted_risk"] >= 0.5).astype(int)

# Keep / reorder columns for display
showcase = showcase[[
    "player_id",
    "injury_date",
    "prior_injuries_count",
    "days_since_last_injury",
    "body_region",
    "diagnosis",
    "label_next_injury_within_Y",
    "predicted_risk",
    "predicted_label"
]]

showcase = showcase.sort_values("predicted_risk", ascending=False)
showcase.head(10)


,player_id,injury_date,prior_injuries_count,days_since_last_injury,body_region,diagnosis,label_next_injury_within_Y,predicted_risk,predicted_label
16936,202695.0,2023-01-20,48,24.0,unknown,unknown,0,0.881054,1
17085,2738.0,2023-03-05,48,57.0,unknown,unknown,0,0.878581,1
16835,202695.0,2022-12-27,47,10.0,unknown,unknown,1,0.877349,1
16528,2738.0,2022-06-08,46,6.0,unknown,unknown,0,0.872760,1
16788,202695.0,2022-12-17,46,7.0,unknown,unknown,1,0.872681,1
16757,202695.0,2022-12-10,45,5.0,unknown,unknown,1,0.867781,1
16527,2738.0,2022-06-02,45,48.0,unknown,unknown,1,0.864245,1
17187,202681.0,2023-03-24,44,7.0,unknown,unknown,0,0.862386,1
16483,2738.0,2022-04-15,44,9.0,unknown,unknown,0,0.862218,1
16718,202695.0,2022-12-05,44,18.0,unknown,unknown,1,0.861458,1
